> 2022.12.16 Ssu-Yun Wang<br/>
[Github @angel870326](https://github.com/angel870326)

# **Manufacturing Data Science Final Project Web**




In [1]:
# sConnect to the Google Drive
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## **Installation & Setup (Flask & Ngrok)**

In [2]:
!pip install flask
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Get your AuthToken from ngrok using this link: https://dashboard.ngrok.com/get-started/your-authtoken

In [3]:
!pip install pyngrok==4.1.1
!ngrok authtoken '1qCRUUcFgnhlSf0o5xS5eilRUUk_7mexMJwXSwcCcPEuvCs2q' # your authtoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


Bootstrap (optional)

In [4]:
!pip install bootstrap-flask

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## **Load Machine Learning Model**

In [5]:
# import pickle
# model = pickle.load(open('model.pkl', 'rb', ))

## **Variables**

In [6]:
import pandas as pd
import numpy as np

In [7]:
# feature_list = [[2, 'Starch Flow'],
#                 [3, 'Amina Flow'],
#                 [5, 'Ore Pulp pH'],
#                 [6, 'Ore Pulp Density'],
#                 [7, 'Flotation Column 01 Air Flow'],
#                 [12, 'Flotation Column 06 Air Flow'],
#                 [16, 'Flotation Column 03 Level'],
#                 [17, 'Flotation Column 04 Level'],
#                 [18, 'Flotation Column 05 Level'],
#                 [20, 'Flotation Column 07 Level']]

In [8]:
# feature_list = [2, 3, 5, 6, 7, 12, 16, 17, 18, 20]

In [9]:
labels = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]

In [30]:
df_demo = pd.DataFrame()
values = []
hint = ""

## **Model**

### (1) Import data

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
from tqdm import tqdm
import time
import joblib
from sklearn.preprocessing import StandardScaler

scaler = joblib.load("/content/gdrive/MyDrive/碩二上/製造數據科學/MDS_Final_Project_Web_v5/static/model/scaler.save") 

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 1.1.3 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [12]:
def make_data(data, timestep = 45, data_type = 'x'):
    query_dim = 2
    if data_type == 'x':
        assert data.ndim == query_dim
        return (np.array([data[i:i+timestep] for i in range(data.shape[0]-(2*timestep))]))
    elif data_type == 'y':
        assert data.ndim == query_dim
        return (np.array([data[i+timestep:i+(2*timestep)] for i in range(data.shape[0]-(2*timestep))]))
    else:
        print('incorrect data type')
        return None

### (2) Dataset & Dataloader

In [13]:
from torch.utils.data import DataLoader, Dataset
class MDSDataset(Dataset):
    def __init__(self, x, y=None, new_y=None):
        self.x = x
        self.y = y
        self.new_y = new_y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        if(self.y is not None and self.new_y is not None):
          return torch.FloatTensor(self.x[idx]), torch.FloatTensor(self.y[idx]), torch.FloatTensor(self.new_y[idx])
        else:
          return torch.FloatTensor(self.x[idx])

In [14]:
from torch.utils.data import DataLoader, Dataset
class MDSDataset(Dataset):
    def __init__(self, x, y=None):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        if(self.y is not None):
          return torch.FloatTensor(self.x[idx]), torch.FloatTensor(self.y[idx])
        else:
          return torch.FloatTensor(self.x[idx])

### (3) Model

#### Encoder

In [15]:
class Encoder(nn.Module):
  def __init__(self, feature_size, hid_dim, n_layers, dropout):
    super().__init__()
    self.n_layers = n_layers
    self.rnn = nn.GRU(feature_size, hid_dim, n_layers, dropout=dropout, batch_first=True, bidirectional=True)
    # self.dropout = nn.Dropout(dropout)

  def forward(self, input):

    # outputs, hidden = self.rnn(torch.from_numpy(input.astype(np.float32)))  # 先轉成 float32 再轉成 tensor
    outputs, hidden = self.rnn(input)
    # print("Encoder outpupt shape:", outputs.shape)
    # print("Encoder output hidden shape:", hidden.shape)
    hidden = hidden.view(self.n_layers, 2, input.shape[0], -1)
    # print("Split two direction hidden state:", hidden.shape)

    # outputs = [batch size, sequence len, hid dim * directions]
    # hidden =  [num_layers * directions, batch size  , hid dim]

    # Concat
    hidden = torch.cat((hidden[:, -2, :, :], hidden[:, -1, :, :]), dim=2)
    # print("Concat hidden state:", hidden.shape)

    # outputs 是最上層RNN的輸出
        
    return outputs, hidden

#### Attention

In [16]:
class Attention(nn.Module):
  def __init__(self, hidden_size):
    super(Attention, self).__init__()
    self.hidden_size = hidden_size
    self.value = nn.Parameter(torch.rand(self.hidden_size * 2))
    self.linear = nn.Linear(self.hidden_size * 2 * 2, self.hidden_size * 2)
    self.atten_weights = None
    self.time_step = 45
  
  def forward(self, encoder_outputs, decoder_hidden):
    # encoder_outputs = [batch size, sequence len, hid dim * directions]
    # decoder_hidden = [num_layers, batch size, hid dim]

    # print("decoder_hidden:", decoder_hidden.shape)
    h = decoder_hidden[0].repeat(self.time_step, 1, 1).transpose(0,1)
    attn_weights = self.dot(encoder_outputs, h) # the dot have done the softmax
    self.atten_weights = attn_weights
    return attn_weights

  def dot(self, encoder_outputs, decoder_hidden):
    # dot product
    # encoder_outputs: BxTxH
    # decoder_hidden: BxH -> BxHx1 # decoder_hidden: NxBxH -> NxBxHx1
    # attention_scores: BxTx1 -> BxT # attention_scores: NxBxTx1 -> NxBxT
    # v: H -> BxH -> BxHx1
    
    # print("encoder_outputs:", encoder_outputs.shape)
    # print("decoder_hidden:", decoder_hidden.shape)
    attn = self.linear(torch.cat([encoder_outputs, decoder_hidden], 2))
    # print("attn:", attn.shape)
    v = self.value.repeat(encoder_outputs.shape[0], 1).unsqueeze(2)
    # print("v:", v.shape)
    attention_scores = torch.bmm(attn, v).squeeze(2)
    # softmax to normalization
    # attention_weights: BxT -> Bx1xT
    attention_weights = F.softmax(attention_scores, dim=1)
    # print("attention_weights:", attention_weights.shape)
    return attention_weights.unsqueeze(1)

#### Decoder 

In [17]:
class Decoder(nn.Module):
  def __init__(self, feature_size, hid_dim, n_layers, dropout, isatt):
    super().__init__()
    self.n_layers = n_layers
    self.hid_dim = hid_dim * 2 # Bidirection 要* 2

    # Attention
    self.isatt = isatt
    self.attention = Attention(hid_dim)

    self.rnn = nn.GRU(self.hid_dim + 1, self.hid_dim, n_layers, dropout=dropout, batch_first=True)
    self.dense1 = nn.Linear(in_features=self.hid_dim ,out_features=self.hid_dim//2)
    self.relu1 = nn.ReLU()
    self.drop1 = nn.Dropout(0.3)

    self.dense2 = nn.Linear(in_features=self.hid_dim//2 ,out_features=self.hid_dim//4)
    self.relu2 = nn.ReLU()
    self.drop2 = nn.Dropout(0.3)
    
    self.dense3 = nn.Linear(in_features=self.hid_dim//4 ,out_features=1)
    # self.dropout = nn.Dropout(dropout)

  def forward(self, input, hidden, encoder_outputs):
    # input = [batch size, vocab size]
    # hidden = [batch size, n layers * directions, hid dim]

    if self.isatt:

      attn = self.attention(encoder_outputs, hidden)
      # print("Attention weights shape:", attn.shape)
      # print("Attention weights :", attn)
      # print("encoder_outputs:", encoder_outputs.shape)
      context_vector = torch.bmm(attn, encoder_outputs)
      # print("context_vector:", context_vector.shape)
      # print("Input:", input.unsqueeze(1).shape)
      rnn_input = torch.cat([context_vector, input.unsqueeze(1)], dim=2) # H+1
      # print("rnn_input:", rnn_input.shape)


    # output, hidden = self.rnn(torch.from_numpy(input.astype(np.float32)).unsqueeze(1), hidden) # RNN 還是要給time_step，所以加一個維度在中間(unsqueeze(1))，代表Decoder一次解碼一個時間維度
    output, hidden = self.rnn(rnn_input, hidden)

    # print("Decoder_rnn_outputs shape:",output.shape) # output = [batch size, 1, hid dim]
    # print("Decoder_rnn_hidden shape", hidden.shape) # hidden = [num_layers, batch size, hid dim]
    
    output = self.dense1(output.squeeze(1))
    # output = self.relu1(output)
    output = self.drop1(output)

    output = self.dense2(output)
    # output = self.relu2(output)
    output = self.drop2(output)

    prediciton = self.dense3(output)
    

    # 將 RNN 的輸出轉為每個詞出現的機率
    # prediction = [batch size, vocab size]
    return prediciton, hidden, attn

#### Seq2seq

In [18]:
import random
class Seq2Seq(nn.Module):
  def __init__(self, encoder, decoder, time_step):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.time_step = time_step
            
  def forward(self, input, target, isInitial=False):
    # input  = [batch size, input len, vocab size]
    # target = [batch size, target len, vocab size]

    # Encoder
    encoder_outputs, encoder_hidden = self.encoder(input) # hidden state already concat

    # preds = []
    preds = torch.zeros(self.time_step, input.shape[0], 1)
    for i in range(0, self.time_step):
      # print("Time:",i+1)

      # decoder input
      decoder_input = target[:, i]


      # print("decoder_input.shape :", decoder_input.shape)  
      if (i == 0): # 第一次用encoder的hidden，因為第一個 decoder 的 hidden state 是由 encoder 傳過來的
        decoder_outputs, decoder_hidden, attn = self.decoder(decoder_input, encoder_hidden, encoder_outputs)
        
      else:      # 之後用自己的hidden
        decoder_outputs, decoder_hidden, attn = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
      
      if(isInitial and i < self.time_step-1):
        target[:, i+1] = decoder_outputs
        
      
      # print("Decoder_outputs shape:",decoder_outputs.shape)
      # print("Decoder_hidden shape", decoder_hidden.shape)
      # preds.append(decoder_outputs.unsqueeze(1))
      preds[i] = decoder_outputs
    # print(type(preds)) 

    if(isInitial):
      return target, attn.reshape(attn.shape[0], attn.shape[2]).tolist()
    else:
      return preds, attn.reshape(attn.shape[0], attn.shape[2]).tolist()

In [19]:
# # training
# batch_size = 128
# time_step = 45
# encoder = Encoder(x_train_shift.shape[2], 128, 3, 0.3)
# decoder = Decoder(y_train_shift.shape[2], 128, 3, 0.3, isatt=True)
# sq2sq_model = Seq2Seq(encoder, decoder, time_step).cuda() # cuda()代表放進GPU裡面準備使用
# sq2sq_model.load_state_dict(torch.load("R10725013_陳志剛 (055).pt"))
# loss = nn.MSELoss()
# optimizer = torch.optim.Adam(sq2sq_model.parameters(), lr=0.001) # optimizer 使用 Adam
# num_epoch = 30

# best_val_loss = 1000

# attn_weights = []
# train_loss_list = []
# val_loss_list = []

# for epoch in range(num_epoch):
  
#   train_loss = 0.0
#   val_loss = 0.0
#   train_updata_times = 0
#   val_updata_times = 0

#   sq2sq_model.train()
#   batch_index = 0 
#   for i, data in enumerate(train_loader):
#     optimizer.zero_grad()

#     teacher_force = random.random()
#     # print(i)
#     if(teacher_force < 0.5 and i != 0 and i != len(train_loader)-1 ): # 頭: 沒有input、尾: batch size問題
#       # print("in")
#       # print("in", i, data[0].shape, last_pred.shape)
#       train_pred, attn_w = sq2sq_model(data[0].cuda(), last_pred.cuda()) # 使用上次的預測結果當作input
#     else:
#       train_pred, attn_w = sq2sq_model(data[0].cuda(), data[1].cuda()) # teach_force

#     if epoch == num_epoch -1: # only save the last weigths
#       attn_weights += attn_w # save the weights

#     # train_pred = sq2sq_model(data[0].cuda(), data[2].cuda())
#     last_pred = torch.tensor(train_pred.transpose(0,1))
#     train_pred = train_pred.transpose(0,1).cuda()
#     # print("!!!!!!", data[1].shape)
#     # print("!!!!!!", train_pred.shape)
#     batch_loss = loss(train_pred, data[1].cuda())
#     # print("Pred:", train_pred[0].detach().cpu().numpy())
#     # print("Ture:", data[1][0].detach().cpu().numpy())
#     batch_loss.backward()
#     optimizer.step()

#     train_loss += batch_loss.item()
#     train_updata_times += 1

#   sq2sq_model.eval()
#   with torch.no_grad():
#     for i, data in enumerate(val_loader):
#       val_pred, attn_w = sq2sq_model(data[0].cuda(), data[1].cuda())
#       val_pred = val_pred.transpose(0,1).cuda()

#       batch_loss = loss(val_pred, data[1].cuda())

#       val_loss += batch_loss.item()
#       val_updata_times += 1

#     temp_best_loss = val_loss/val_updata_times
#     if(temp_best_loss < best_val_loss):
#       best_val_loss = temp_best_loss
#       torch.save(sq2sq_model.state_dict(), "R10725013_陳志剛.pt")

#     train_loss_list.append(train_loss/train_updata_times)
#     val_loss_list.append(val_loss/val_updata_times)
#     #將結果 print 出來
#     print("Epoch:", str(epoch+1)+"/"+str(num_epoch), "Train loss:", train_loss/train_updata_times, "Val loss:", val_loss/val_updata_times)

In [20]:
# import matplotlib.pyplot as plt
# x = [i for i in range(num_epoch)]
# train_loss_list = []
# val_loss_list = []

# plt.plot(x, train_loss_list)
# plt.xlabel("Epoch")
# plt.ylabel("train loss")

# plt.title("Training")
# plt.show()

# plt.plot(x, val_loss_list)
# plt.xlabel("Epoch")
# plt.ylabel("valid loss")

# plt.title("Validation")
# plt.show()

### (4) GA

In [21]:
class GeneticAlgorithm():
  def __init__(self, data_1, data_2, upper_bound, population_size:int = 100, crossover_rate:float = 1, mutate_rate:float = 0.02):
    self.features_name = {1: 'Starch Flow',
                          2: 'Amina Flow',
                          3: 'Ore Pulp pH',
                          4: 'Ore Pulp Density',
                          5: 'Flotation Column 01 Air Flow',
                          6: 'Flotation Column 06 Air Flow',
                          7: 'Flotation Column 03 Level',
                          8: 'Flotation Column 04 Level',
                          9: 'Flotation Column 05 Level',
                          10: 'Flotation Column 07 Level'}
    self.features_id = {1:2, 2:3, 3:5, 4:6, 5:7, 6:12, 7:16, 8:17, 9:18, 10:20}
    self.threshold = upper_bound
    
    self.data_1 = data_1
    self.data_2 = data_2

    self.population_size = population_size
    self.crossover_rate = crossover_rate
    self.mutate_rate = mutate_rate

    self.population = {}
    self.seen = set()

    self.best_fitness = np.inf
    self.best_solution = None


  @staticmethod
  def encode(sloution: list) -> str:
    return '_'.join([str(i) for i in sloution])


  @staticmethod
  def decode(sloution: str) -> list:
    return [int(i) for i in sloution.split('_')]


  def get_fitness(self, solution: list, data_1: torch.Tensor, data_2:torch.Tensor) -> float:
    # clone and 
    data_1 = data_1.clone()
    data_1 = scaler.inverse_transform(data_1[0])

    # reformat_solution
    solution_dict = {i:0 for i in range(1, 11)}
    for unit in solution:
      if unit != 0:
        solution_dict[abs(unit)] += unit/abs(unit)
    
    # check maximum and update data
    for key, value in solution_dict.items():
      original_value = data_1[-1][self.features_id[key]]
      data_1[-1][self.features_id[key]] *= 1+solution_dict[key]/100

    data_1 = torch.FloatTensor(scaler.transform(data_1).reshape(1, 45, 21))
    # calculate score
    test_pred, _ = sq2sq_model2(data_1.cuda(), data_2.cuda())
    score = test_pred[-1][0].item()
    return score


  def initialize(self):
    while len(self.population) < self.population_size:
      solution = [random.randint(-10, 10) for i in range(10)]
      solution.sort()
      encode_solution = self.encode(solution)
      
      if encode_solution not in self.seen:
        fitness = self.get_fitness(solution, self.data_1, self.data_2)
        self.population[encode_solution] = fitness
        self.seen.add(encode_solution)

        if fitness < self.best_fitness:
          self.best_fitness = fitness
          self.best_solution = encode_solution

  @staticmethod
  def crossover(solution_1: list, solution_2: list) -> list:
    random.shuffle(solution_1)
    random.shuffle(solution_2)

    new_sloution = solution_1[:5] + solution_2[5:]

    return new_sloution

  def mutate(self, solution: list) -> list:
    for i, unit in enumerate(solution):
      if random.random() < self.mutate_rate:
        solution[i] = random.randint(-10, 10)
    solution.sort()
    return solution

  def generate(self):
    next_generation = {}
    new_population = {}
    best_solution = None
    best_fitness = np.inf

    solutions, weights = zip(*self.population.items())
    weights = 1 / np.array(weights, dtype=float)
    probs = weights / sum(weights)

    while len(next_generation) < self.population_size:
      # crossover
      if random.random() < self.crossover_rate:
        parent1, parent2 = np.random.choice(solutions, 2, p=probs)
        child = self.crossover(self.decode(parent1), self.decode(parent2))
        
        # mutate
        child = self.mutate(child)

        encode_child = self.encode(child)
        if encode_child not in self.seen:
          fitness = self.get_fitness(child, self.data_1, self.data_2)

          next_generation[encode_child] = fitness
          self.seen.add(encode_child)

          if fitness < best_fitness:
              best_solution = encode_child
              best_fitness = fitness

    # update best fitness
    if best_fitness < self.best_fitness:
      self.best_solution = best_solution
      self.best_fitness = best_fitness

    # selection
    while len(new_population) < self.population_size:
        solution1 = random.choice(list(self.population.keys()))
        solution2 = random.choice(list(next_generation.keys()))
        fitness_1 = self.population.pop(solution1)
        fitness_2 = next_generation.pop(solution2)
        if fitness_1 < fitness_2:
            new_population[solution1] = fitness_1
        else:
            new_population[solution2] = fitness_2
    
    self.population = new_population
  
  def translate(self, solution: str):
    suggest = {i:0 for i in self.features_name.values()}
    
    for unit in self.decode(solution):
      if unit > 0:
        suggest[self.features_name[abs(unit)]] += 1
      elif unit < 0:
        suggest[self.features_name[abs(unit)]] -= 1

    hint = ''
    for (feature, adjustment) in suggest.items():
      hint+= f"{feature}: {adjustment}%\n"

    return hint

  def optimize(self, max_iteration=5, early_stopping=True):
    start_time = time.time()
    current_best = self.best_fitness
    patience = 0

    for i in range(max_iteration):
      self.generate()
      if self.best_fitness < current_best:
        current_best = self.best_fitness
        patience = 0
      else:
        patience+=1

      if early_stopping and (patience==5 or self.best_fitness < self.threshold):
        break

    hint = self.translate(self.best_solution)
    return self.best_fitness, hint

### (5) Optimization



In [22]:
# Load model
batch_size = 128
time_step = 45
encoder = Encoder(21, 128, 3, 0.3)
decoder = Decoder(21, 128, 3, 0.3, True) 
sq2sq_model2 = Seq2Seq(encoder, decoder, time_step).cuda()
sq2sq_model2.load_state_dict(torch.load("/content/gdrive/MyDrive/碩二上/製造數據科學/MDS_Final_Project_Web_v5/static/model/model.pt")) #改路徑

<All keys matched successfully>

In [23]:
def predictor(df: pd.DataFrame) -> tuple:
  x_val_shift = make_data(df.iloc[:, :-1], timestep = 45, data_type = 'x')
  y_val_shift = make_data(df.iloc[:, -1:], timestep = 45, data_type = 'y')
  val_set = MDSDataset(x_val_shift, y_val_shift)
  val_loader = DataLoader(val_set, batch_size = 1, shuffle=False)

  original = []
  new = []
  upper_bound = 4.5
  hint = None

  sq2sq_model2.eval()
  with torch.no_grad():
    for i, (data_1, data_2) in enumerate(tqdm(val_loader)):
      if(i == 0):
        val_pred, attn_w = sq2sq_model2(data_1.cuda(), data_2.cuda())
        val_decoder_input = torch.FloatTensor(val_pred.transpose(0,1))
        score = val_pred[-1][0].item()
        original.append(score)
        new.append(score)
      else:
        val_pred, attn_w = sq2sq_model2(data_1.cuda(), val_decoder_input.cuda())
        score = val_pred[-1][0].item()
        original.append(score)
        if score > upper_bound:
          GA = GeneticAlgorithm(data_1, val_decoder_input, upper_bound)
          GA.initialize()
          new_score, hint = GA.optimize(max_iteration=5, early_stopping=False)
          new.append(new_score)
          break
        else:
          new.append(score)
        val_decoder_input = torch.FloatTensor(val_pred.transpose(0,1))
 
  return original, new, hint

In [24]:
def adjuster(df: pd.DataFrame, adjustments: list) -> list:
  x_val_shift = make_data(df.iloc[:, :-1], timestep = 45, data_type = 'x')
  y_val_shift = make_data(df.iloc[:, -1:], timestep = 45, data_type = 'y')
  val_set = MDSDataset(x_val_shift, y_val_shift)
  val_loader = DataLoader(val_set, batch_size = 1, shuffle=False)

  features_id = [2, 3, 5, 6, 7, 12, 16, 17, 18, 20]
  scores = []

  sq2sq_model2.eval()
  with torch.no_grad():
    for i, (data_1, data_2) in enumerate(tqdm(val_loader)):
      if(i == 0):
        val_pred, attn_w = sq2sq_model2(data_1.cuda(), data_2.cuda())
        val_decoder_input = torch.FloatTensor(val_pred.transpose(0,1))
      else:
        if i == len(val_loader) -1:
          data_1 = scaler.inverse_transform(data_1[0])
          for j, adjustment in enumerate(adjustments):
            data_1[-1][features_id[j]] *= 1+adjustment/100
          data_1 = torch.FloatTensor(scaler.transform(data_1).reshape(-1, data_1.shape[0], data_1.shape[1]))

        val_pred, attn_w = sq2sq_model2(data_1.cuda(), val_decoder_input.cuda())
        val_decoder_input = torch.FloatTensor(val_pred.transpose(0,1))
          

      score = val_pred[-1][0].item()
      scores.append(score)

  return scores

## **Run the Web**

In [25]:
from flask_ngrok import run_with_ngrok
from flask_bootstrap import Bootstrap5
from flask import Flask, render_template, redirect, request, session, send_file, url_for, Response, stream_with_context
import os
from werkzeug.utils import secure_filename
import json

In [35]:
template_path = '/content/gdrive/MyDrive/碩二上/製造數據科學/MDS_Final_Project_Web_v5/templates'
static_path = '/content/gdrive/MyDrive/碩二上/製造數據科學/MDS_Final_Project_Web_v5/static'

# Downlaod file
download_path = os.path.join(static_path, 'downloads')

# Upload file
upload_folder = os.path.join(static_path, 'uploads') # Define folder to save uploaded files to process further
ALLOWED_EXTENSIONS = {'csv'} # Define allowed files

app = Flask(__name__, template_folder = template_path, static_folder=static_path)
app.config['UPLOAD_FOLDER'] = upload_folder # Configure upload file path flask
app.secret_key = 'MDS Final Project Group 4' # Define secret key to enable session

bootstrap = Bootstrap5(app)
run_with_ngrok(app)   #starts ngrok when the app is run

@app.route("/", methods=['GET'])
def home():
    return render_template('index.html')

@app.route("/download")
def downloadFile():
    download_path = os.path.join(static_path, 'downloads/template.csv')
    return send_file(download_path, as_attachment=True)
 
@app.route('/', methods=['GET', 'POST'])
def uploadFile():
    if request.method == 'POST':
        # upload file flask
        uploaded_df = request.files['uploaded-file']
        # Extracting uploaded data file name
        data_filename = secure_filename(uploaded_df.filename)
        # flask upload file to database (defined uploaded folder in static path)
        uploaded_df.save(os.path.join(app.config['UPLOAD_FOLDER'], data_filename))
        # Storing uploaded file path in flask session
        session['uploaded_data_file_path'] = os.path.join(app.config['UPLOAD_FOLDER'], data_filename)
        return render_template('index.html')

@app.route('/predict', methods=['GET', 'POST'])
def predict():
    global df_demo, values, hint
    if df_demo.empty:
        df_demo = pd.read_csv(os.path.join(static_path, 'uploads/df_demo.csv'), index_col='Unnamed: 0')
        values, values_adj, hint = predictor(df_demo)
    
    values = [round(num, 2) for num in values]
    print("Orig.", values)
    results_dict = {"labels": labels, "values": values}

    hint_item = hint.split('\n')
    anomaly_hint = hint_item[0]
    for i in hint_item[1:]:
        anomaly_hint = anomaly_hint + ", " + i

    return render_template('predict.html', results_dict=json.dumps(results_dict), anomaly=anomaly_hint)

@app.route('/adjust', methods=['GET', 'POST'])
def adjust():
    # Get slider values
    slider_value = []
    for i in range(1,11):
        slider_value.append(int(request.form.get('slider{}'.format(i))))

    adjusted_features = {"f1":slider_value[0], "f2":slider_value[1], "f3":slider_value[2], "f4":slider_value[3], "f5":slider_value[4], "f6":slider_value[5], "f7":slider_value[6], "f8":slider_value[7], "f9":slider_value[8], "f10":slider_value[9]}
    print("Slider", slider_value)

    # Adjustment
    values_adj = adjuster(df_demo, slider_value)
    values_adj = [round(num, 2) for num in values_adj]
    print("Adj.", values_adj)

    results_dict = {"labels": labels, "values": values, "values_adj": values_adj}

    return render_template('adjust.html', results_dict=json.dumps(results_dict), adjusted_features=adjusted_features)

@app.route('/readjust', methods=['GET'])
def readjust():
    return redirect('predict')

 
if __name__=='__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://afde-35-198-250-9.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [17/Dec/2022 15:50:28] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Dec/2022 15:50:29] "GET /static/css/styles.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Dec/2022 15:50:29] "GET /static/js/scripts.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Dec/2022 15:50:30] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Dec/2022 15:50:31] "GET /static/assets/img/bg-masthead.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Dec/2022 15:50:31] "GET /static/js/scripts.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Dec/2022 15:50:31] "GET /static/css/styles.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Dec/2022 15:50:34] "GET /static/assets/favicon.ico HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Dec/2022 15:50:34] "GET /static/assets/img/bg-masthead.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Dec/2022 15:50:44] "GET /predict HTTP/1.1" 200 -


[3.5, 2.97, 3.14, 3.18, 3.19, 3.22, 3.24, 3.32, 3.96, 4.76]


...

[3.5, 2.97, 3.14, 3.18, 3.19, 3.22, 3.24, 3.32, 3.96, 4.76]
[-1, -1, 4, 0, 0, 0, 0, -1, 1, 0]


100%|██████████| 10/10 [00:00<00:00, 32.40it/s]
INFO:werkzeug:127.0.0.1 - - [17/Dec/2022 15:53:07] "POST /adjust HTTP/1.1" 200 -


[3.5, 2.97, 3.14, 3.18, 3.19, 3.22, 3.24, 3.32, 3.96, 3.81]


INFO:werkzeug:127.0.0.1 - - [17/Dec/2022 15:53:17] "GET /readjust HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [17/Dec/2022 15:53:17] "GET /predict HTTP/1.1" 200 -


[3.5, 2.97, 3.14, 3.18, 3.19, 3.22, 3.24, 3.32, 3.96, 4.76]
